# 第2单元: 使用FrozenLake-v1 ⛄️ 和Taxi-v3 🚕 进行 Q-Learning
在这份笔记中, **你将从头开始编写你的第一个强化学习智能体**使用Q-Learning来玩冰冻湖 ❄️ 并将其分享到社区, 请记得实验不同的配置

❓如果你有任何问题, 请在discord的#study-group-unit2频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境: 
* [FrozenLake-v1](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)
* [Taxi-v3](https://www.gymlibrary.ml/environments/toy_text/taxi/)

📚 强化学习库: Python和NumPy

⬇️ 这是**你将在几分钟内实现的目标**的示例. ⬇️

![envs.gif](./assets/envs.gif)

## 这份笔记的目标🏆

在这份笔记学习结束后, 你将:

* 能够使用环境库**Gym**.
* 能够从头开始编写一个Q-Learning智能体.
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub**.

## 这份笔记来自深度强化学习课程
![深度强化学习课程.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:

* 📖 研究深度强化学习的**理论和实践**.
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ 在独特的环境中训练**智能体**.

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗

在深入研究笔记之前, 你需要:

🔲 📚 [阅读第2单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit2/README.md)

🔲 📚 [阅读**Q-Learning简介的第1部分**](https://huggingface.co/blog/deep-rl-q-part1)

🔲 📚 [阅读**Q-Learning简介的第2部分**](https://huggingface.co/blog/deep-rl-q-part2)

🔲 📢 注册我们的Discord服务器并在#introduce-yourself频道介绍自己 🥳

🔲 🐕 你是Discord新手吗? 请查看我们的discord 101以获得最佳实践 👉 https://github.com/huggingface/deep-rl-class/blob/main/DISCORD.Md

## 一个Q-Learning的小回顾

* Q-Learning是一种**强化学习算法**
    * 通过训练一个包含**动作(action)-价值(value)的Q-函数(Q-Function)**, Q-表(Q-Table)作为内部存储器, 它包含了**所有的状态-动作对.**
    * 给定一个状态和动作, 我们的Q-函数将**搜索Q-表对应的值.**

![Q-函数-2.jpg](./assets/Q-function-2.jpg)

* 当训练完成时, **我们得到一个最优的Q-函数, 因此得到一个最优的Q-表.**
* 如果我们有**最优的Q-函数**, 那我们就知道**对于每个状态最好的行动是什么**, 所以我们就有一个最优策略.

![link-value-policy.jpg](./assets/link-value-policy.jpg)

但是一开始, 我们的**Q-表是无效的, 因为它的每个状态-动作对都是任意值(大多数时候我们将Q-表初始化为0).** 当我们探索环境并更新我们的Q-表时, 它将给我们越来越好的近似值.

![Q-learning-1.jpg](./assets/Q-learning-1.jpg)

这是Q-Learning的伪代码:

![Q-learning-2.jpg](./assets/Q-learning-2.jpg)

### 第0步: 设置虚拟屏幕 🖥

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境(记录视频帧).**

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [ ]:
!apt install gitlfs ffmpeg 
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

In [ ]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### 第1步: 安装依赖项 🔽

第一步是安装多个依赖项:

* `gym`: 包含FrozenLake-v1 ⛄️ 和Taxi-v3 🚕 环境.
* `pygame`: 用于FrozenLake-v1和Taxi-v3的UI.
* `numpy`: 用于处理我们的Q-表.

Hugging Face Hub 🤗 是一个任何人都可以分享和探索模型和数据集的地方. 它有版本控制, 评估, 可视化和其他功能, 可以允许你简单地与他人协作.

你可以在这看到全部可用的深度强化学习模型. 👉 https://huggingface.co/models?other=q-learning

In [ ]:
!pip install gym==0.24.0 pygame
!pip install huggingface_hub
!pip install imageio

### 第2步: 导入包 📦

除了安装的库, 我们还使用:
* `imageio`: 生成回放视频.

In [ ]:
import gym
import imageio
import numpy as np
import pickle5 as pickle

我们现在已经准备好编写我们的Q-Learing算法了 🔥

# 第1部分: 冰冻湖 ⛄️ (不打滑版)

### 第1步: 创建并理解[冰冻湖环境 ⛄️ ](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)
---
💡 当你开始使用一个环境的好习惯是查看它的文档.

👉 https://www.gymlibrary.ml/environments/toy_text/frozen_lake/

---
我们将训练我们的Q-Learning智能体从起始状态(S)导航到目标状态(G), 方法是仅在冰面(F)上行走并避开冰窟窿(H).

我们有两个尺寸的环境:
* `map_name='4x4'`: 4x4地图版本.
* `map_name='8x8'`: 8x8地图版本.

环境有两种模式:
* `is_slippery=False`: 由于冰冻湖不打滑的特性, 智能体总能沿预期方向移动.
* `is_slippery=True`: 由于冰冻湖打滑(随机)的特性, 智能体可能不总沿预期方向移动.

现在让我们使用4x4地图和不打滑的版本.

In [ ]:
# 创建FrozenLake-v1环境并使用4x4地图和不打滑的版本.
env = gym.make()

### 答案

In [ ]:
env = gym.make(id='FrozenLake-v1',
               map_name='4x4',
               is_slippery=False)

你也可以自定义地图来使用:
```python
desc=['SFFF', 'FHFH', 'FFFH', 'HFFG']
gym.make('FrozenLake-v1', desc=desc, is_slippery=True)
```
但是我们现在将使用默认的环境.

### 让我们一起看看环境是什么样的:

In [ ]:
# 我们使用 gym.make('<环境的名称>') 创建环境.
env.reset()
print('_' * 5 + '可观察的环境' + '_' * 5)
print()
print('可观察的环境', env.observation_space)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

我们通过`可观察的环境 Discrete(16)`看到, 可观察的环境是一个表示**智能体当前位置current_row * nrows + current_col(行和列均从0开始)的值.**

例如, 在4x4的地图上目标位置可以被计算成: 3 * 4 + 3 = 15. 可能的环境值取决于地图的大小. **例如, 4x4的地图有16个可能的环境值.**

对于当前, 初始值 = 0

![image.png](./assets/image.png)

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5)
print()
print('动作的总数', env.action_space.n)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

动作空间(智能体可用的动作集)是有4个可用动作的离散值 🎮:

* 0: 向左走
* 1: 向下走
* 2: 向右走
* 3: 向上走

奖励函数 💰:

* 到达目标: +1
* 到达冰窟窿: 0
* 到达冰面: 0

### 第2步: 创建并初始化Q-表 🗄
(👀 伪代码的第1步)
![Q-learning-2.jpg](./assets/Q-learning-2.jpg)
是时候初始化我们的Q-表了! 要知道我们需要使用多少行(states)和列(actions), 我们需要知道可观察和动作空间. OpenAI Gym给我们提供了一个方法: `env.action_space.n`和`env.observation_space.n`

In [ ]:
state_space = 
print('一共有', state_space, '个可能的状态')

action_space = 
print('一共有', action_space, '个可能的动作')

In [ ]:
# 让我们创建大小为(state_space, action_space)的Q-表, 并使用np.zeros将它们的值初始化为0.
def initialize_q_table(state_space, action_space):
    Qtable = 
    return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

### 答案

In [ ]:
state_space = env.observation_space.n
print('一共有', state_space, '个可能的状态')

action_space = env.action_space.n
print('一共有', action_space, '个可能的动作')

In [ ]:
# 让我们创建大小为(state_space, action_space)的Q-表, 并使用np.zeros将它们的值初始化为0.
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros([state_space, action_space])
    return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

### 第3步: 定义epsilon-greedy策略 🤖️
epsilon-greedy是一个平衡探索/经验的训练策略.

epsilon-greedy的思路是:
* 当概率为$1-\epsilon$: **我们使用经验**(也就是我们的智能体选择一个最高的状态-动作对值的动作).
* 当概率为$\epsilon$: **我们进行探索**(尝试随机动作).

然后随着训练的进行, 我们逐渐**降低epsilon的值, 因为我们需要越来越少的探索和使用越来越多的经验.**

![Q-learning-4.jpg](./assets/Q-learning-4.jpg)

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
    # 随机生成一个在0和1之前的数字.
    random_num =
    # 如果random_num > epsilon --> 使用经验.
    if random_num > epsilon:
        # 从给定状态中采取最大值的动作;
        # 在这np.argmax()非常有用.
        action =
    # 否则 --> 探索
    else:
        action = # 进行随机动作.

    return action

### 答案

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
    # 随机生成一个在0和1之前的数字.
    random_num = np.random.uniform(0, 1)
    # 如果random_num > epsilon --> 使用经验.
    if random_num > epsilon:
        # 从给定状态中采取最大值的动作;
        # 在这np.argmax()非常有用.
        action = np.argmax(Qtable[state])
    # 否则 --> 探索
    else:
        action = env.action_space.sample() # 进行随机动作.

    return action

### 第4步: 定义贪心策略 🤖️
请记住我们有两个策略, 因为Q-Learning是一种**off-policy**算法. 这意味着我们使用**不同的策略来执行动作和更新价值函数.**
* epsilon-greedy策略(动作策略)
* 贪心策略(更新策略)

贪心策略也将是我们训练Q-learing智能体后最终的策略. 贪心策略用来从Q-表中选择一个动作.

![off-on-4.jpg](./assets/off-on-4.jpg)

In [ ]:
def greedy_policy(Qtable, state):
    # 经验: 从给定状态中采取最大值的动作.
    action =

    return action

### 答案

In [ ]:
def greedy_policy(Qtable, state):
    # 经验: 从给定状态中采取最大值的动作.
    action = np.argmax(Qtable[state])

    return action

### 第5步: 定义超参数 ⚙️
与探索相关的参数是一些最重要的超参数.
* 我们需要确保我们的智能体**探索足够的状态空间**来学习一个好的值近似(value approximation), 为此我们需要逐渐降低epsilon.
* 如果你epsilon减小得过快(太高的衰减率), **你将承担智能体卡住的风险**, 因为你的智能体没有探索足够的状态空间, 因此无法解决问题.

In [ ]:
# 训练参数.
n_training_episodes = 10000  # 训练的总轮数.
learning_rate = 0.7  # 学习率.

# 评估参数.
n_eval_episodes = 100  # 测试的总轮数.

# 环境参数.
env_id = 'FrozenLake-v1'  # 环境的名称.
max_steps = 99  # 每轮的最大步数.
gamma = 0.95  # 衰减系数.
eval_seed = []  # 环境的评估种子.

# 探索参数.
epsilon = 1.0  # 探索率.
max_epsilon = 1.0  # 初始探索率.
min_epsilon = 0.05  # 最小探索率.
decay_rate = 0.0005  # 指数探索衰减率.

### 第6步: 创建训练循环

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in range(n_training_episodes):
        # 减小epsilon(因为我们需要越来越少的探索).
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # 重置环境.
        state = env.reset()

        # 循环.
        for step in range(max_steps):
            # 使用epsilon-greedy策略选择动作At.
            action =

            # 采取动作At并观察Rt+1和St+1,
            # 采取动作(a)并观察结果状态(s')和奖罚(r).
            new_state, reward, done, info =

            # 更新Q(s, a) = Q(s, a) + lr[R(s, a) + gamma * max Q(s', a') - Q(s, a)]
            Qtable[state][action] =

            # 如果done=True, 退出循环.
            if done:
                break

            # 更新状态.
            state = new_state

    return Qtable

### 答案

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in range(n_training_episodes):
        # 减小epsilon(因为我们需要越来越少的探索).
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        # 重置环境.
        state = env.reset()

        # 循环.
        for step in range(max_steps):
            # 使用epsilon-greedy策略选择动作At.
            action = epsilon_greedy_policy(Qtable, state, epsilon)

            # 采取动作At并观察Rt+1和St+1,
            # 采取动作(a)并观察结果状态(s')和奖罚(r).
            new_state, reward, done, info = env.step(action)

            # 更新Q(s, a) = Q(s, a) + lr[R(s, a) + gamma * max Q(s', a') - Q(s, a)]
            Qtable[state][action] += learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            # 如果done=True, 退出循环.
            if done:
                break

            # 更新状态.
            state = new_state

    return Qtable

### 第7步: 训练Q-Learning智能体 🏃

In [ ]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)    

### 第8步: 现在让我们看看Q-表是什么样子 👀

In [ ]:
Qtable_frozenlake

### 第9步: 定义评估函数 📝

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
    """用`n_eval_episodes`轮评估智能体, 并返回奖励的均值和标准差.

    Args:
        env: 评估环境.
        max_steps: 每轮的最大步数.
        n_eval_episodes: 测试的总轮数.
        Q: Q-表.
        seed: 评估随机种子数组(用于Taxi-v3).

    Returns:
        奖励的均值和标准差.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        if seed:
            state = env.reset(seed=seed[episode])
        else:
            state = env.reset()
        step = 0
        done = False
        total_rewards_ep = 0

        for step in range(max_steps):
            # 在给定状态下, 采取具有最大期望奖励的动作(索引).
            action = np.argmax(Q[state])
            new_state, reward, done, info = env.step(action)
            total_rewards_ep += reward

            if done:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)

    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

### 第10步: 评估我们的Q-Learing智能体 📈
* 正常情况你的奖励均值应该是1.0
* 这相对容易, 因为状态空间确实非常小(16). 你可以尝试[使用打滑版替换.](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/)

In [ ]:
# 评估我们的智能体.
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f'平均奖励={mean_reward:.2f} +/- {std_reward:.2f}')

### 第11步(不涉及核心内容, 可选): 发布我们训练好的模型到 Hub 上 🔥

现在我们看到经过训练之后得到了很棒的结果, 我们可以通过一行代码发布我们训练的模型到hub🤗上.

这有一个模型卡的例子:

![image.png](./assets/image1.png)

在底层, Hub使用基于git的仓库(即使你不知道什么是git也不用担心), 这意味着你可以在实验和提高你的智能体以后更新新版本的模型.

#### 请勿修改下面的代码

In [ ]:
%%capture
import datetime
import json
from pathlib import Path

from huggingface_hub import HfApi, HfFolder, Repository
from huggingface_hub.repocard import metadata_eval_result, metadata_save

In [ ]:
def record_video(env, Qtable, out_directory, fps=1):
    images = []  
    done = False
    state = env.reset(seed=random.randint(0, 500))
    img = env.render(mode='rgb_array')
    images.append(img)
    
    while not done:
        # 在给定状态下, 采取具有最大期望奖励的动作(索引).
        action = np.argmax(Qtable[state])
        state, reward, done, info = env.step(action) # 我们直接使用next_state = state来记录顺序(recording logic).
        img = env.render(mode='rgb_array')
        images.append(img)
        
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id,
                model,
                env,
                video_fps=1,
                local_repo_path='hub',
                commit_message='Push Q-Learning agent to Hub',
                token=None):
    _, repo_name = repo_id.split('/')

    eval_env = env

    # Step 1: Clone or create the repo
    # Create the repo (or clone its content if it's nonempty)
    api = HfApi()

    repo_url = api.create_repo(repo_id=repo_id,
                               token=token,
                               private=False,
                               exist_ok=True)

    # Git pull
    repo_local_path = Path(local_repo_path) / repo_name
    repo = Repository(repo_local_path, clone_from=repo_url, use_auth_token=True)
    repo.git_pull()

    repo.lfs_track(['*.mp4'])

    # Step 1: Save the model
    if env.spec.kwargs.get('map_name'):
        model['map_name'] = env.spec.kwargs.get('map_name')
        if env.spec.kwargs.get('is_slippery', '') == False:
            model['slippery'] = False

    print(model)

    # Pickle the model
    with open(Path(repo_local_path) / 'q-learning.pkl', 'wb') as f:
        pickle.dump(model, f)

    # Step 2: Evaluate the model and build JSON
    mean_reward, std_reward = evaluate_agent(eval_env, model['max_steps'], model['n_eval_episodes'], model['qtable'],
                                             model['eval_seed'])

    # First get datetime
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
        'env_id': model['env_id'],
        'mean_reward': mean_reward,
        'n_eval_episodes': model['n_eval_episodes'],
        'eval_datetime': eval_form_datetime,
    }
    # Write a JSON file
    with open(Path(repo_local_path) / 'results.json', 'w') as outfile:
        json.dump(evaluate_data, outfile)

    # Step 3: Create the model card
    # Env id
    env_name = model['env_id']
    if env.spec.kwargs.get('map_name'):
        env_name += '-' + env.spec.kwargs.get('map_name')

    if env.spec.kwargs.get('is_slippery', '') == False:
        env_name += '-' + 'no_slippery'

    metadata = {}
    metadata['tags'] = [
        env_name,
        'q-learning',
        'reinforcement-learning',
        'custom-implementation'
    ]

    # Add metrics
    eval = metadata_eval_result(model_pretty_name=repo_name,
                                task_pretty_name='reinforcement-learning',
                                task_id='reinforcement-learning',
                                metrics_pretty_name='mean_reward',
                                metrics_id='mean_reward',
                                metrics_value=f'{mean_reward:.2f} +/- {std_reward:.2f}',
                                dataset_pretty_name=env_name,
                                dataset_id=env_name)

    # Merges both dictionaries
    metadata = {**metadata, **eval}

    model_card = f'''
      # **Q-Learning** Agent playing **{env_id}**
      This is a trained model of a **Q-Learning** agent playing **{env_id}** .
      '''

    model_card += '''
      ## Usage
      ```python
      '''

    model_card += f'''model = load_from_hub(repo_id='{repo_id}', filename='q-learning.pkl')
    
      # Don't forget to check if you need to add additional attributes (is_slippery=False etc)
      env = gym.make(model['env_id'])
    
      evaluate_agent(env, model['max_steps'], model['n_eval_episodes'], model['qtable'], model['eval_seed'])
      '''

    model_card += '''
      ```
      '''

    readme_path = repo_local_path / 'README.md'
    readme = ''
    if readme_path.exists():
        with readme_path.open('r', encoding='utf8') as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open('w', encoding='utf-8') as f:
        f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)

    # Step 4: Record a video
    video_path = repo_local_path / 'replay.mp4'
    record_video(env, model['qtable'], video_path, video_fps)

    # Push everything to hub
    print(f'Pushing repo {repo_name} to the Hugging Face Hub')
    repo.push_to_hub(commit_message=commit_message)

    print(f'Your model is pushed to the hub. You can view your model here: {repo_url}')